In [1]:
from typing import Tuple

import itertools

import pandas

from prophet import Prophet
from prophet.diagnostics import cross_validation, performance_metrics
from prophet.serialize import model_to_json, model_from_json

from sklearn.metrics import mean_squared_error

/home/ankitapurv/.conda/envs/env-prophet/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.


In [2]:
country = 'italy'

df = pandas.read_excel(io = f'../../../data/processed/{country}.xlsx')

In [3]:
def make_dataset(df_processed: pandas.DataFrame, df_covid_measures: pandas.DataFrame = pandas.DataFrame()) -> Tuple[pandas.DataFrame, pandas.DataFrame]:
    df_mrd = df_processed[['Time', 'Unemployment_Rate_TOT']].rename(
        columns = {'Time': 'ds', 'Unemployment_Rate_TOT': 'y'}
    )
    df_mrd = df_mrd.drop(index = df_mrd[pandas.isnull(df_mrd['y'])].index, inplace = False)
    return df_mrd, df_covid_measures

In [4]:
df_mrd, _ = make_dataset(df)

In [5]:
def train_test_split(df_mrd: pandas.DataFrame, test_size: int = 12) -> Tuple[pandas.DataFrame, pandas.DataFrame]:
    df_test = df_mrd.tail(test_size)
    df_train = df_mrd.drop(index = df_mrd.tail(test_size).index, inplace = False)
    return df_train, df_test

In [6]:
df_train, df_test = train_test_split(df_mrd, 12)

In [7]:
def train_model(df: pandas.DataFrame, param_grid: dict, eval_metric: str = 'rmse', eval_metric_optimize: str = 'min') -> Prophet:
    # Generate all combinations of parameters
    all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
    eval_metric_values = []  # Store the eval_metric_values for each params here

    # Use cross validation to evaluate all parameters
    for params in all_params:
        m = Prophet(**params).fit(df)  # Fit model with given params
        df_cv = cross_validation(m, horizon = '60 days', parallel="processes")
        df_p = performance_metrics(df_cv, rolling_window=1, monthly=True)
        eval_metric_values.append(df_p[eval_metric].values[0])

    # Find the best parameters
    tuning_results = pandas.DataFrame(all_params)
    tuning_results[eval_metric] = eval_metric_values

    tuning_results.sort_values(by = eval_metric, axis = 'index', ascending = True, inplace = True)
    best_params = {}
    if eval_metric_optimize == 'min':
        best_params = tuning_results.iloc[0].to_dict()
    elif eval_metric_optimize == 'max':
        best_params = tuning_results.iloc[-1].to_dict()
    else:
        raise ValueError("Invalid Parameter Value: param 'eval_metric_optimize' may only have values 'min' or 'max'.")


    optimal_model = Prophet(**params).fit(df)
    return optimal_model

In [8]:
param_grid = {  
    'changepoint_prior_scale': [0.001, 0.01, 0.1, 0.5],
    'seasonality_prior_scale': [0.01, 0.1, 1.0, 10.0],
}

model = train_model(df_train, param_grid)

01:27:55 - cmdstanpy - INFO - Chain [1] start processing
01:27:55 - cmdstanpy - INFO - Chain [1] done processing
01:27:55 - cmdstanpy - ERROR - Chain [1] error: error during processing Communication error on send
Optimization terminated abnormally. Falling back to Newton.
01:27:55 - cmdstanpy - INFO - Chain [1] start processing
01:27:57 - cmdstanpy - INFO - Chain [1] done processing
01:27:57 - cmdstanpy - INFO - Chain [1] start processing
01:27:57 - cmdstanpy - INFO - Chain [1] start processing
01:27:57 - cmdstanpy - INFO - Chain [1] start processing
01:27:57 - cmdstanpy - INFO - Chain [1] start processing
01:27:57 - cmdstanpy - INFO - Chain [1] start processing
01:27:57 - cmdstanpy - INFO - Chain [1] start processing
01:27:57 - cmdstanpy - INFO - Chain [1] start processing
01:27:57 - cmdstanpy - INFO - Chain [1] start processing
01:27:57 - cmdstanpy - INFO - Chain [1] done processing
01:27:57 - cmdstanpy - INFO - Chain [1] done processing
01:27:57 - cmdstanpy - INFO - Chain [1] done p

In [9]:
def test_model(df_test: pandas.DataFrame, model: Prophet) -> Tuple[pandas.DataFrame, float]:
    "return predicted values and rmse"
    df_predicted: pandas.DataFrame = model.predict(df_test)
    rmse: float = mean_squared_error(y_true = df_test['y'], y_pred = df_predicted['yhat'], squared = False)
    return df_predicted, rmse

In [10]:
df_predicted, rmse = test_model(df_test, model)

In [11]:
df_predicted

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2022-03-01,8.716397,8.379441,9.044868,8.716397,8.716397,0.000048,0.000048,0.000048,0.000048,0.000048,0.000048,0.0,0.0,0.0,8.716445
1,2022-04-01,8.672726,8.343422,9.013808,8.672726,8.672726,-0.006864,-0.006864,-0.006864,-0.006864,-0.006864,-0.006864,0.0,0.0,0.0,8.665861
2,2022-05-01,8.630463,8.321225,9.005820,8.630463,8.630463,0.035157,0.035157,0.035157,0.035157,0.035157,0.035157,0.0,0.0,0.0,8.665620
3,2022-06-01,8.586792,8.232312,8.938645,8.586792,8.592527,0.003590,0.003590,0.003590,0.003590,0.003590,0.003590,0.0,0.0,0.0,8.590382
4,2022-07-01,8.544529,8.140347,8.881096,8.539940,8.569157,-0.033796,-0.033796,-0.033796,-0.033796,-0.033796,-0.033796,0.0,0.0,0.0,8.510733
5,2022-08-01,8.500858,8.079788,8.825798,8.475951,8.545770,-0.057180,-0.057180,-0.057180,-0.057180,-0.057180,-0.057180,0.0,0.0,0.0,8.443678
6,2022-09-01,8.457187,8.129990,8.900431,8.415859,8.537855,0.040289,0.040289,0.040289,0.040289,0.040289,0.040289,0.0,0.0,0.0,8.497476
7,2022-10-01,8.414925,8.109350,8.896107,8.345736,8.521340,0.082782,0.082782,0.082782,0.082782,0.082782,0.082782,0.0,0.0,0.0,8.497707
8,2022-11-01,8.371253,8.038651,8.862661,8.259742,8.504958,0.079307,0.079307,0.079307,0.079307,0.079307,0.079307,0.0,0.0,0.0,8.450560
9,2022-12-01,8.328991,7.971149,8.809434,8.173013,8.502070,0.034986,0.034986,0.034986,0.034986,0.034986,0.034986,0.0,0.0,0.0,8.363977


In [12]:
rmse

0.4779043351839792

In [13]:
with open(f'{country}_prophet_optimized_model.json', 'w') as f:
    f.write(model_to_json(model))

In [14]:
df_future = pandas.DataFrame(data = {'ds': ['2023-03-01', '2023-04-01', '2023-05-01']})

In [15]:
df_future_prediction: pandas.DataFrame = model.predict(df_future)

In [16]:
df_future_prediction

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2023-03-01,8.202203,7.890209,8.563840,8.202203,8.202203,0.025313,0.025313,0.025313,0.025313,0.025313,0.025313,0.0,0.0,0.0,8.227516
1,2023-04-01,8.158532,7.817619,8.513808,8.158532,8.158532,-0.010572,-0.010572,-0.010572,-0.010572,-0.010572,-0.010572,0.0,0.0,0.0,8.147960
2,2023-05-01,8.116270,7.839751,8.561547,8.116270,8.116270,0.057673,0.057673,0.057673,0.057673,0.057673,0.057673,0.0,0.0,0.0,8.173943
